In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

from sksurv.metrics import concordance_index_censored
from lifelines.datasets import load_rossi

data = load_rossi()
data.dropna(inplace=True)
print(data.shape)
data.head()

(432, 9)


,week,arrest,fin,age,race,wexp,mar,paro,prio
0,20,1,0,27,1,0,0,1,3
1,17,1,0,18,1,0,0,1,8
2,25,1,0,19,0,1,0,1,13
3,52,0,1,23,1,1,1,1,1
4,52,0,0,19,0,1,0,1,3


In [2]:
# TODO Experiments:
# - Performance of global model on data aggregated across clients 
# - Performance of global model on separate local datasets 
# - Performance of local models on separate local datasets 

In [3]:
event_col = "arrest"
duration_col = "week"

In [4]:
from utils.client import Client 

client = Client(data, n_knots=5, n_epochs=5, event_col=event_col, duration_col=duration_col)
client.preprocess_data()
client.init_model()

2025-04-28 07:55:46.419867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
N_CLIENTS = 3 

data_idx = np.array_split(np.arange(data.shape[0]), N_CLIENTS)

participants = [] 
for idx in data_idx:
    # Init client objects with mutually exclusive data samples 
    client = Client(data.iloc[idx], n_knots=5, n_epochs=5, event_col=event_col, duration_col=duration_col)
    # Apply data pre-processing steps 
    client.preprocess_data()
    # Initialize model and parameters 
    client.init_model()

    participants.append(client)

In [6]:
for client in participants:
    client.fit_model()

In [13]:
all_beta, all_gamma = [], [] 
for client in participants:
    client.fit_model()
    all_beta.append(client.model.beta)
    all_gamma.append(client.model.gamma)

In [20]:
np.mean(all_beta, axis=0) 

array([[-0.05866686, -0.35781536,  0.02991288,  0.0512445 , -0.5335343 ,
        -0.01136155,  0.14971308]], dtype=float32)

In [21]:
np.mean(all_gamma, axis=0) 

array([[0.64689  , 1.2909883, 2.0879104, 3.016133 , 3.3963156, 3.6230056,
        3.7182686]], dtype=float32)